# Experimentacion

In [ ]:
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

df = pd.read_csv("expectativa_de_vida.csv")

#### Como la cantidad de filas (paises) en el dataset no es demasiado significativa (miles y miles de elementos), no hare un train set que intente ser un fiel representante de todos los datos de la tabla. Sino que utilizare a todos los paises del dataset para realizar experimentos. 
#### En algunos casos, en vez de incluir a todos, me centrare en ciertos grupos en especifico, excluyendo a toda la muestra. Pero esto sera con la intencion de querer demostrar cierta anomalia o dato interesante que nos puedan proveer los datos. Como ejemplo, vere como segmentar y emplear regresion solo con paises encasillados en la condicion de 'Developing' y los que se encuentran en 'Developed'. Tambien habra un modelo con todos los paises, y otro donde no haya outliers. Se trataran a los outliers con suficiente detenimiento para entender su condicion, por que se salen tanto de la norma, son creibles sus datos? O simplemente estan encasillados en una categoria que no les corresponde si se la compara con el resto de su informacion?

In [2]:
df

,Unnamed: 0,Country,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status
0,0,Afghanistan,58.19375,269.0625,78.2500,0.014375,34.960110,64.562500,2362.2500,15.51875,...,8.252500,52.3125,0.10000,340.015425,9.972260e+06,16.58125,15.58125,0.415375,8.21250,Developing
1,1,Albania,75.15625,45.0625,0.6875,4.848750,193.259091,98.000000,53.3750,49.06875,...,5.945625,98.0625,0.10000,2119.726679,6.969116e+05,1.61875,1.70000,0.709875,12.13750,Developing
2,2,Algeria,73.61875,108.1875,20.3125,0.406667,236.185241,78.000000,1943.8750,48.74375,...,4.604000,91.8750,0.10000,2847.853392,2.164983e+07,6.09375,5.97500,0.694875,12.71250,Developing
3,3,Angola,49.01875,328.5625,83.7500,5.740667,102.100268,70.222222,3561.3125,18.01875,...,3.919333,47.6875,2.36875,1975.143045,1.014710e+07,6.19375,6.66875,0.458375,8.04375,Developing
4,4,Antigua and Barbuda,75.05625,127.5000,0.0000,7.949333,1001.585226,98.266667,0.0000,38.42500,...,4.791333,98.3125,0.12500,9759.305728,NaN,3.42500,3.37500,0.488625,8.84375,Developing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,188,Venezuela (Bolivarian Republic of),73.38750,163.0000,9.3750,7.420000,0.000000,66.250000,165.0000,54.48750,...,4.998667,68.5000,0.10000,NaN,NaN,1.65000,1.56250,0.726812,12.78750,Developing
179,189,Viet Nam,74.77500,126.5625,29.1875,3.087333,0.000000,87.538462,4232.9375,11.18750,...,5.977333,91.7500,0.14375,NaN,NaN,14.92500,15.62500,0.627063,11.51250,Developing
180,190,Yemen,63.86250,211.8125,39.3750,0.047333,0.000000,55.687500,2761.1875,33.48750,...,5.005333,72.6250,0.10000,NaN,NaN,13.83125,13.75000,0.475500,8.50625,Developing
181,191,Zambia,53.90625,354.3125,33.4375,2.239333,89.650407,69.818182,6563.8125,17.45000,...,5.824000,74.2500,11.93125,811.811841,6.260246e+06,6.88125,6.76250,0.498437,11.21250,Developing


### Gracias al analisis exploratorio de datos provisto en 'EDA.ipynb' se puede descartar la siguiente data:

#### <b>Features 
 - **Hepatitis B**: Se correlaciona mucho con Polio y Diphteria
 - **Alcohol**: No parece ser un indicador que ayude a explicar la expectativa de vida
 - **GDP**: Puede ser reemplazado por Percentage Expenditure
 - **Population**: Se puede reemplazar por Infant Deaths y Under-Five Deaths
 - **Income Composition of Resources y Schooling**: Ambos correlacionan fuerte con BMI

#### <b> Paises
 - **Sudan, Sudan del Sur, Corea del Norte y Somalia**: Todos ellos tienen data incompleta

### Saco a los paises nombrados mas arriba del dataset

In [86]:
df_new = df[(df["Country"] != "South Sudan")] # Borro a Sudan del Sur

In [87]:
df_new = df_new[(df_new["Country"] != "Sudan")] # Borro a Sudan

In [88]:
df_new = df_new[(df_new["Country"] != "Democratic People's Republic of Korea")] # Borro a Corea del Norte

In [89]:
df_new = df_new[(df_new["Country"] != "Somalia")] # Borro a Somalia

In [103]:
print(df.shape) # Con todos los paises
print(df_new.shape) # Sin esos paises (TENGO QUE HACER MENOS REPETITIVA LA ELIMINACION DE PAISES > BUSCAR EN GOOGLE)

(183, 22)
(179, 22)


### Ya tengo mi muestra sin los paises que puedan llegar a truncar la experimentacion. Ahora me encargo de tener tres muestras distintas: Los paises de status 'Developing', los de estado 'Developed', y todas las naciones juntas.

In [96]:
df_developed = df_new[(df_new["Status"] == "Developed")] # Los developed
df_developing = df_new[(df_new["Status"] == "Developing")] # Los developing
# df_new es el que contiene a todas las naciones juntas

In [102]:
print(df_developed.shape)
print(df_developing.shape)

(32, 22)
(147, 22)


## Busco variables para mis muestras de regresion: Empiezo con el data set que contiene a todos los paises